In [ ]:
import numpy as np
import pandas as pd
import requests
import mason_functions as mf

# Acquire Exercises

1. Using the code from the lesson as a guide and the REST API from https://python.zgulde.net/api/v1/items as we did in the lesson, create a dataframe named items that has all of the data for items.

In [ ]:
#get response using requests library
response = requests.get('https://python.zgulde.net/api/v1/items')

#assign json data to variable
data = response.json()

#verify data type
print(type(data))

#verify data
data

In [ ]:
#check api keys
data.keys()

In [ ]:
#assign dataframe to data acquired
items = pd.DataFrame(data['payload']['items'])

#check it
items.info()

In [ ]:
#get response utilizing requests library from Zach's API
response = requests.get('https://python.zgulde.net/api/v1/items?page=2')

#assign variable to json data
data = response.json()

#concatenate previous dataframe with second-page data
items = pd.concat([items, pd.DataFrame(data['payload']['items'])])

#check it
items.info()

In [ ]:
#get response from Zach's API using requests library
response = requests.get('https://python.zgulde.net/api/v1/items?page=3')

#assign variable to json data
data = response.json()

#concatenate previous dataframe with new data from 3rd page
items = pd.concat([items, pd.DataFrame(data['payload']['items'])])

#check it
items.info()

In [ ]:
#checking
items.head()

In [ ]:
#change max rows to a high number for viewability
pd.options.display.max_rows = 69

#view dataframe
items

In [ ]:
#assign missing values to numbers 0 to 9
#items.loc[40:, 'index'] = range(0, 10)

In [ ]:
#verify
#items.loc[40:]

2. Do the same thing, but for stores (https://python.zgulde.net/api/v1/stores)

In [ ]:
#get a response from Zach's API using the requests library
response = requests.get('https://python.zgulde.net/api/v1/stores')

#assign variable to json data
data = response.json()

#verify data
data

There is only one page here.

In [ ]:
#assign variable to datafarme
stores = pd.DataFrame(data['payload']['stores'])
stores

In [ ]:
#checking
stores.info()

3. Extract the data for sales (https://python.zgulde.net/api/v1/sales). There are a lot of pages of data here, so your code will need to be a little more complex. Your code should continue fetching data from the next page until all of the data is extracted.

In [ ]:
#request response from api using requests module
response = requests.get('https://python.zgulde.net/api/v1/sales')

#assign variable to json data
data = response.json()

#verify data
data

In [ ]:
#generate dataframe with json data
sales = pd.DataFrame(data['payload']['sales'])
sales

In [ ]:
#assign variable to max page number
max_page = data['payload']['max_page']
max_page

In [ ]:
#commence loop
for n in range(2, max_page + 1):
    response = requests.get(f'https://python.zgulde.net/api/v1/sales?page={n}')
    data = response.json()
    mein_sales = pd.DataFrame(data['payload']['sales'])
    sales = pd.concat([sales, mein_sales])

In [ ]:
sales

In [ ]:
#check it
sales.info()

4. Save the data in your files to local csv files so that it will be faster to access in the future.

In [ ]:
#use pandas .to_csv method to write dataframes into .csv's
items.to_csv('zachs_items.csv')
stores.to_csv('zachs_stores.csv')
sales.to_csv('zachs_sales.csv')

5. Combine the data from your three separate dataframes into one large dataframe.

In [ ]:
#merge first two tables with an inner join based on key columns
join = pd.merge(sales,
                items,
                how = 'inner',
                left_on = 'item',
                right_on = 'item_id'
               )
#check it
join.head()

In [ ]:
#merge the merged table and the stores table using the store number as a foreign key
super_frame = pd.merge(join,
                       stores,
                       how = 'inner',
                       left_on = 'store',
                       right_on = 'store_id'
                      )
#checking
super_frame.head()

In [ ]:
#checking
super_frame.info()

In [ ]:
#checking again
super_frame

6. Acquire the Open Power Systems Data for Germany, which has been rapidly expanding its renewable energy production in recent years. The data set includes country-wide totals of electricity consumption, wind power production, and solar power production for 2006-2017. You can get the data here: https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv

In [ ]:
#read csv link with pandas .read_csv function
power = pd.read_csv('https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv')

#check it
power.head()

In [ ]:
#checking
power.info()

7. Make sure all the work that you have done above is reproducible. That is, you should put the code above into separate functions in the acquire.py file and be able to re-run the functions and get the same data.

In [ ]:
def get_zachs_pages(payload):
    
    #get response from api
    response = requests.get(f'https://python.zgulde.net/api/v1/{payload}?page=1')
    data = response.json()
    df = pd.DataFrame(data['payload'][payload])
    
    #assign variable to max page number
    max_page = data['payload']['max_page']
    
    #commence loop
    for n in range(2, max_page + 1):
        response = requests.get(f'https://python.zgulde.net/api/v1/{payload}?page={n}')
        data = response.json()
        df_0 = pd.DataFrame(data['payload'][payload])
        df = pd.concat([df, df_0])
    
    #return finished frame
    return df

In [ ]:
#testing
items = get_zachs_pages('items')
items

In [ ]:
#testing
stores = get_zachs_pages('stores')
stores

In [ ]:
import os

In [ ]:
def get_items():
    
    #set up if-conditional to see if a .csv is available
    if os.path.isfile('zachs_items.csv'):
        
        #if there is, read the data into a dataframe
        items = pd.read_csv('zachs_items.csv', index_col = 0)
    else:
        #get items
        items = get_zachs_pages('items')
        
        #write data to frame
        items = pd.DataFrame(items)
        
        #cache data in .csv
        items.to_csv('zachs_items.csv')
    
    return items

In [ ]:
def get_stores():
    
    #set up if-conditional to see if a .csv is available
    if os.path.isfile('zachs_stores.csv'):
        
        #if there is, read the data into a dataframe
        stores = pd.read_csv('zachs_stores.csv', index_col = 0)
    else:
        #get stores
        stores = get_zachs_pages('stores')
        
        #write data to frame
        stores = pd.DataFrame(stores)
        
        #cache data in .csv
        stores.to_csv('zachs_stores.csv')
    
    return stores

In [ ]:
def get_sales():
    
    #set up if-conditional to see if a .csv is available
    if os.path.isfile('zachs_sales.csv'):
        
        #if there is, read the data into a dataframe
        sales = pd.read_csv('zachs_sales.csv', index_col = 0)
    else:
        #get sales
        sales = get_zachs_pages('sales')
        
        #write data to frame
        sales = pd.DataFrame(sales)
        
        #cache data in .csv
        sales.to_csv('zachs_sales.csv')
    
    return sales

In [ ]:
def super_store_frame(items, stores, sales):
    
    #merge first two tables with an inner join based on key columns
    join = pd.merge(sales,
                    items,
                    how = 'inner',
                    left_on = 'item',
                    right_on = 'item_id'
                   )
    
    #merge the merged table and the stores table using the store number as a foreign key
    super_frame = pd.merge(join,
                           stores,
                           how = 'inner',
                           left_on = 'store',
                           right_on = 'store_id'
                          )
    
    return super_frame

In [ ]:
def get_power():
    
    #read .csv with pandas .read_csv function
    power = pd.read_csv('https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv')

    #return df
    return power